In [ ]:
%pip install sentence-transformers qdrant-client

In [9]:
import csv
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient, models, conversions
from tqdm.notebook import tqdm

In [10]:
client = QdrantClient(path="vec.db")

In [11]:
encoder = SentenceTransformer("all-MiniLM-L6-v2") #embedding model

/workspace/ABCFinance/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
client.recreate_collection(
    collection_name="investopedia",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

/tmp/ipykernel_4456/1457044021.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [15]:
import datasets

ds = datasets.load_dataset("openvega-simon/investopedia", split="train")

In [16]:
from langchain.docstore.document import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["md_content"], metadata={"title": doc["title"], "source": doc["url"]})
    for doc in tqdm(ds)
]

  0%|          | 0/4723 [00:00<?, ?it/s]

In [17]:
points = []
for idx, doc in tqdm(enumerate(RAW_KNOWLEDGE_BASE), total=len(RAW_KNOWLEDGE_BASE)):
    content = f'''{doc.metadata["title"]}: {doc.page_content}'''
    vector = encoder.encode(content).tolist()
    payload = {"title": doc.metadata["title"], "source": doc.metadata["source"], "page_content": doc.page_content}
    point = models.PointStruct(id=idx, vector=vector, payload=payload)
    points.append(point)

  0%|          | 0/4723 [00:00<?, ?it/s]

In [1]:
points

NameError: name 'points' is not defined

In [19]:
client.upload_points(
    collection_name="investopedia",  # Replace with your actual collection name
    points=points,
    parallel=4,
)

In [20]:
points = client.search(
    collection_name="investopedia",
    query_vector=encoder.encode("J Term").tolist(),
    limit=3,
)

In [21]:
points

[ScoredPoint(id=0, version=0, score=0.6032048648035324, payload={'title': 'J', 'source': 'https://www.investopedia.com/terms/j/j.asp', 'page_content': "## What Is J?\n\n\n The term J refers to a designation for [Nasdaq](https://www.investopedia.com/terms/n/nasdaq.asp)-listed stocks that specifies that the stock has voting rights. The designation appears as the fifth letter following a dot after a stock's four-letter [ticker symbol](https://www.investopedia.com/terms/s/stocksymbol.asp). It is added to denote a shareholder vote situation. The letter J is a temporary suffix that is removed once the shareholder vote situation is resolved. Other letter designations are used to describe [share classes](https://www.investopedia.com/terms/s/share_class.asp), foreign issues, preferred issues, and a company's financial status.\n\n\n\n\n### Key Takeaways\n\n\n* J is a fifth-letter designation used to illustrate that a Nasdaq-listed security has voting rights.\n* Nasdaq-listed stocks have four cha